In [ ]:
# Installation
%pip install fastapi uvicorn    # API
%pip install numpy              # Efficient data handling
%pip install python-dotenv      # .env => extracting hidden info
%pip install requests           # Performing API calls

# Imports
from fastapi import FastAPI, HTTPException
from dotenv import load_dotenv
import os
import requests
from enum import Enum

In [ ]:
# Configuration
load_dotenv()

# Global Variables
DB_URL = os.getenv("DATABASE_URL")
class ConsumerField(Enum):
    VISITS = 1
    ORDERS = 2
    FAVORITED = 3
    FRIENDS = 4
    ALL = 5

In [ ]:
app = FastAPI()

In [ ]:
# Pull consumer information from the DB
async get_consumer_info(field):
    response = await requests.get(f"{DB_URL}/consumer")
    if not response.ok:
        raise HTTPException(status_code=424, detail="Failed to pull consumer information from DB")
    consumer_data = await response.json()

    # Extract desired information
    match field:
        case ConsumerField.VISITS:
            return consumer_data['restaurant_visits']
        case ConsumerField.ORDERS:
            return consumer_data['orders']
        case ConsumerField.FAVORITED:
            return consumer_data['favorite_restaurants']
        case ConsumerField.FRIENDS:
            all_friends = consumer_data['friendships_a'] | consumer_data['friendships_b']
            return all_friends
        case ConsumerField.ALL:
            return consumer_data
        case _: # default case
            return consumer_data

# Pull restaurant information from the DB
async get_restaurant_info():
    response = await requests.get(f"{DB_URL}/restaurant")
    if not response.ok:
        raise HTTPException(status_code=424, detail="Failed to pull restaurant information from DB")
    restaurant_data = await response.json()
    return restaurant_data

In [ ]:
# Features & default weights
# TODO: Add actual weights (currently arbitrarily)
# Feature Weights
rating_weight = 0.3; avg_cost_weight = 0.2; distance_weight = 0.125
category_budget = 1 - rating_weight - avg_cost_weight - distance_weight
num_categories = 29
category_weight = category_budget / num_categories
feature_weights = {
    'rating': rating_weight,
    'avg_cost': avg_cost_weight,
    'distance': distance_weight,
    # Following are categories one-hot encoding
    'asian': category_weight,
    'bakery': category_weight,
    'barfood': category_weight,
    'bbq': category_weight,
    'breakfast': category_weight,
    'burgers': category_weight,
    'cafe': category_weight,
    'chinese': category_weight,
    'desserts': category_weight,
    'fastfood': category_weight,
    'french': category_weight,
    'greek': category_weight,
    'healthy': category_weight,
    'indian': category_weight,
    'italian': category_weight,
    'japanese': category_weight,
    'korean': category_weight,
    'latinamerican': category_weight,
    'mediterranean': category_weight,
    'mexican': category_weight,
    'middleeastern': category_weight,
    'pizza': category_weight,
    'salads': category_weight,
    'sandwiches': category_weight,
    'seafood': category_weight,
    'sushi': category_weight,
    'thai': category_weight,
    'vegan': category_weight,
    'vegetarian': category_weight,
    'vietnamese': category_weight,
}

# Existing Restaurant Interaction Weights
# NOTE: Used to define how to weigh restaurant's to generate consumer vector
# TODO: Edit later?
restaurant_type_weights = {
    'survey': 0.6,
    'liked': 0.8,
    'order': 0.4,
    'visit': 0.2,
}

# Consumer Vector Feedback/Interaction Weights
# NOTE: Used to define feedback on updating consumer vector based on interactions with recommendations
# TODO: Edit later?
recommendation_feedback_weights = {
    'liked': 0.2,
    'order': 0.1,
    'visit': 0.04,
}

In [ ]:
## Utility Functions
# Winsorization is used to solve min-max normalization isues caused by outliers by taking, say, the 1st percentile & 99th percentile rather than min & max
# Resource: https://medium.com/@whyamit404/implementing-pandas-winsorize-ad1e51ec548b
def winsorization(arr):
    np_arr = np.array(arr)

    winsorized_min = np.percentile(arr, 1)
    winsorized_max = np.percentile(arr, 99)

    return [winsorized_min, winsorized_max]

In [ ]:
# Consumer Vector Operations
# TODO: 
def generate_init_consumer_vector():
    # TODO: 
def update_consumer_vector():
    # TODO: 
    

In [ ]:
# Restaurants Vector Generation
# TODO: 
async def generate_restaurant_vectors():
    # TODO: 
    restaurant_data = await get_restaurant_info()

    # Find average values (for default & normalization)
    # NOTE: Extracted array of average ratings inspired by https://stackoverflow.com/questions/50216362/how-to-extract-from-a-json-array-in-python
    ratings_arr = [restaurant['avg_rating'] for restaurant in restaurant_data]
    rating_sum = sum(rating for rating in ratings_arr if (rating != -1)) # Only use ratings from restaurants w/ existing ratings
    num_ratings = sum(1 for rating in ratings_arr if (rating != -1)) # Only count ratings from restaurants w/ existing ratings
    avg_rating =  rating_sum / num_ratings 

    cost_arr = [restaurant['avg_cost'] for restaurant in restaurant_data]
    avg_cost = sum(cost_arr) / len(cost_arr)
    min_winsorized_cost, max_winsorized_cost = winsorization(cost_arr)


    # Generate restaurant vectors
    restaurant_vectors_dict = {} # Restaurant ID to vector mappings
    for restaurant in restaurant_data:
        # Normalized Values

        rating_normalized = (restaurant['avg_rating'] if (restaurant['avg_rating'] == -1 ) else default_rating) / 5.0
        # Min-Max Normalization w/ Inversion (lower cost is more favorable)
        cost_normalized_inverted = 1 - ( ( restaurant['avg_cost'] - min_winsorized_cost ) / ( max_winsorized_cost - min_winsorized_cost )) 
        
        restaurant_vector = np.array([
            rating_normalized,
            cost_normalized_inverted,
            # TODO: Add distance (conditionally, based on if the field exists)
            # TODO: Add categories
            
        ])

        restaurant[restaurant['restaurant_id']] = # TODO:


In [ ]:
# Cosine similarity
# TODO: 

In [ ]:
# Return top-N recommendations
# TODO: 

In [ ]:
@app.get("/recommend/{consumer_id}")
async def recommend(consumer_id: int):
    # TODO: Recommendation code to be called here
    await ...